In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=0ecf2ac4d8d06867560388dafd51577cbb4a96ea20f6bb3521d06054578c2840
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.metrics import accuracy_score, f1_score
import lime
from lime.lime_text import LimeTextExplainer

from operator import itemgetter
from tqdm import tqdm

In [ ]:
def load_data(data_file):
  # read csv file
  df = pd.read_csv(data_file)

  # replace nan(no value) comment with ""(empty string)
  df.fillna("", inplace=True)

  tweets = df['Tweet'].tolist()
  labels = df['Party'].tolist()

  labels = [0 if label == "Democrat" else 1 for label in labels]

  return tweets, labels

In [ ]:
tweets, labels = load_data('/content/drive/MyDrive/Datasets/Tweets/ExtractedTweets.csv')

In [ ]:
tweets = np.array(tweets)
labels = np.array(labels)

len(tweets)

86460

In [ ]:
new_tweets = tweets[35370:86460]
new_labels = labels[35370:86460]

n = int(len(new_tweets) / 65)
x = [tweets[i:i + n] for i in range(0, len(new_tweets), n)]
y = [labels[i:i + n] for i in range(0, len(new_labels), n)]

first_half_tweets = x[0]
second_half_tweets = x[1]
third_half_tweets = x[2]
fourth_half_tweets = x[3]
fifth_half_tweets = x[4]
sixth_half_tweets = x[5]
seventh_half_tweets = x[6]
eighth_half_tweets = x[7]
ninth_half_tweets = x[8]
tenth_half_tweets = x[9]
eleventh_half_tweets = x[10]
twelveth_half_tweets = x[11]
thirteenth_half_tweets = x[12]
fourteenth_half_tweets = x[13]
fifteen_half_tweets = x[14]
sixteen_half_tweets = x[15]
seventeen_half_tweets = x[16]
eighteen_half_tweets = x[17]
nineteen_half_tweets = x[18]
twenty_half_tweets = x[19]
twentyone_half_tweets = x[20]
twentytwo_half_tweets = x[21]
twentythree_half_tweets = x[22]
twentyfour_half_tweets = x[23]
twentyfive_half_tweets = x[24]
twentysix_half_tweets = x[25]
twentyseven_half_tweets = x[26]
twentyeight_half_tweets = x[27]
twentynine_half_tweets = x[28]
thirty_half_tweets = x[29]
thirtyone_half_tweets = x[30]
thirtytwo_half_tweets = x[31]
thirtythree_half_tweets = x[32]
thirtyfour_half_tweets = x[33]
thirtyfive_half_tweets = x[34]
thirtysix_half_tweets = x[35]
thirtyseven_half_tweets = x[36]
thirtyeight_half_tweets = x[37]
thirtynine_half_tweets = x[38]
# fourty_half_tweets = x[39]
# fourtyone_half_tweets = x[40]
# fourtytwo_half_tweets = x[41]
# fourtythree_half_tweets = x[42]
# fourtyfour_half_tweets = x[43]





first_half_labels = y[0]
second_half_labels = y[1]
third_half_labels = y[2]
fourth_half_labels = y[3]
fifth_half_labels = x[4]
sixth_half_labels = x[5]
seventh_half_labels = x[6]
eighth_half_labels = x[7]
ninth_half_labels = x[8]
tenth_half_labels = x[9]
eleventh_half_labels = x[10]
twelveth_half_labels = x[11]
thirteenth_half_labels = x[12]
fourteenth_half_labels = x[13]
fifteen_half_labels = x[14]
sixteen_half_labels = x[15]
seventeen_half_labels = x[16]
eighteen_half_labels = x[17]
nineteen_half_labels = x[18]
twenty_half_labels = x[19]
twentyone_half_labels = x[20]
twentytwo_half_labels = x[21]
twentythree_half_labels = x[22]
twentyfour_half_labels = x[23]
twentyfive_half_labels = x[24]
twentysix_half_labels = x[25]
twentyseven_half_labels = x[26]
twentyeight_half_labels = x[27]
twentynine_half_labels = x[28]
thirty_half_labels = x[29]
thirtyone_half_labels = x[30]
thirtytwo_half_labels = x[31]
thirtythree_half_labels = x[32]
thirtyfour_half_labels = x[33]
thirtyfive_half_labels = x[34]
thirtysix_half_labels = x[35]
thirtyseven_half_labels = x[36]
thirtyeight_half_labels = x[37]
thirtynine_half_labels = x[38]
# fourty_half_labels = x[39]
# fourtyone_half_labels = x[40]
# fourtytwo_half_labels = x[41]
# fourtythree_half_labels = x[42]
# fourtyfour_half_labels = x[43]

In [ ]:
# Tokenize and pad sequences
max_words = 50000
max_len = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(tweets)
sequences = tokenizer.texts_to_sequences(tweets)
tweets = pad_sequences(sequences, maxlen=max_len)


In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=42)

In [ ]:
# Check if GPU is available and set memory growth to avoid issues
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Define the CNN model
embedding_dim = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)


Epoch 1/3
1730/1730 [==============================] - 26s 14ms/step - loss: 0.4624 - accuracy: 0.7655 - val_loss: 0.3672 - val_accuracy: 0.8233
Epoch 2/3
1730/1730 [==============================] - 15s 9ms/step - loss: 0.2655 - accuracy: 0.8848 - val_loss: 0.3828 - val_accuracy: 0.8306
Epoch 3/3
1730/1730 [==============================] - 14s 8ms/step - loss: 0.1470 - accuracy: 0.9417 - val_loss: 0.5038 - val_accuracy: 0.8212


In [ ]:
# Predict on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")


541/541 [==============================] - 1s 2ms/step


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1:.4f}')


Accuracy: 0.8162
F1 Score: 0.8232


In [ ]:
# Save the model
model.save('/content/drive/MyDrive/Datasets/Tweets/Saved Model/tweet_classification_model_cnn.h5')
print("Model saved to tweet_classification_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to tweet_classification_model.h5


In [ ]:
# Define prediction function
def predict_party(tweet, tokenizer, model, max_len):
    sequence = tokenizer.texts_to_sequences([tweet])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded_sequence)[0][0]
    return 'Democrat' if prediction < 0.5 else 'Republican'

In [ ]:
# print(predict_party(tweets[0], tokenizer, model, max_len))

1/1 [==============================] - 0s 280ms/step
Democrat


In [ ]:
# Define a function to predict probabilities
def predict_proba(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return np.array([[1 - pred[0], pred[0]] for pred in model.predict(padded_sequences)])

In [ ]:
# Create a LIME explainer
explainer = LimeTextExplainer(class_names=['Democrat', 'Republican'])


In [ ]:
# Function to explain predictions
def explain_prediction(tweet):
    explanation = explainer.explain_instance(tweet, predict_proba, num_features=10)
    return explanation

In [ ]:
# Display the explanation
# exp.show_in_notebook(text=True)

In [ ]:
def sort_tuples_array_by_second_item(tuples):
    # Sort the tuples by the second item using the itemgetter function
    return sorted(tuples, key=itemgetter(1))

In [ ]:
def get_max_explained_words(txt):

  prediction = predict_party(model, txt, tokenizer, max_len)
  # print(" ")
  # print("prediction")
  # print(prediction)

  exp = explain_prediction(txt)

  exp_list = []
  for x in zip(exp.local_exp[1], exp.as_list()):
    exp_list.append((x[1][0], x[1][1], x[0][0]))

  # print("exp_list")
  # print(exp_list)

  # features with negative score are for Male class
  democrat_list = list(filter(lambda x: x[1] < 0, exp_list))
  democrat_list = sort_tuples_array_by_second_item(democrat_list)

  # print("democrat_list")
  # print(democrat_list)
  # print(len(democrat_list))

  # features with positive score are for female class
  republican_list = list(filter(lambda x: x[1] > 0, exp_list))
  republican_list = sort_tuples_array_by_second_item(republican_list)

  # print("republican_list")
  # print(republican_list)
  # print(len(republican_list))

  # min is used while the democrat score is negative
  democrat_mc = min(democrat_list, key=itemgetter(1)) if len(democrat_list) else None

  # print("democrat_mc")
  # print(democrat_mc)

  # max is used while the republican score is negative
  republican_mc = max(republican_list, key=itemgetter(1)) if len(republican_list) else None

  # print("republican_mc")
  # print(republican_mc)

  # if comment predicted Male
  if prediction == "Democrat":
    if len(democrat_list) > 1:
      democrat_mc = democrat_list[0]
      if (democrat_mc, 0) in words:
        words[(democrat_mc[0], 0)]['lime_score'].extend(democrat_mc[1])
        words[(democrat_mc[0], 0)]['position'] = democrat_mc[2]
      else:
        words[(democrat_mc[0], 0)] = {}
        words[(democrat_mc[0], 0)]['lime_score'] = [democrat_mc[1]]
        words[(democrat_mc[0], 0)]['position'] = democrat_mc[2]
        wordsForCSV.append([democrat_mc[0], 0, democrat_mc[1]])

      democrat_mc = democrat_list[1]
      if (democrat_mc, 0) in words:
        words[(democrat_mc[0], 0)]['lime_score'].extend(democrat_mc[1])
        words[(democrat_mc[0], 0)]['position'] = democrat_mc[2]
      else:
        words[(democrat_mc[0], 0)] = {}
        words[(democrat_mc[0], 0)]['lime_score'] = [democrat_mc[1]]
        words[(democrat_mc[0], 0)]['position'] = democrat_mc[2]
        wordsForCSV.append([democrat_mc[0], 0, democrat_mc[1]])
    elif len(democrat_list) == 1:
      democrat_mc = democrat_list[0]
      if (democrat_mc, 0) in words:
        words[(democrat_mc[0], 0)]['lime_score'].extend(democrat_mc[1])
        words[(democrat_mc[0], 0)]['position'] = democrat_mc[2]
      else:
        words[(democrat_mc[0], 0)] = {}
        words[(democrat_mc[0], 0)]['lime_score'] = [democrat_mc[1]]
        words[(democrat_mc[0], 0)]['position'] = democrat_mc[2]
        wordsForCSV.append([democrat_mc[0], 0, democrat_mc[1]])

  else:
    if len(republican_list) > 1:
      republican_mc = republican_list[(len(republican_list)-1)]
      if (republican_mc, 1) in words:
        words[(republican_mc[0], 1)]['lime_score'].extend(republican_mc[1])
        words[(republican_mc[0], 1)]['position'] = republican_mc[2]
      else:
        words[(republican_mc[0], 1)] = {}
        words[(republican_mc[0], 1)]['lime_score'] = [republican_mc[1]]
        words[(republican_mc[0], 1)]['position'] = republican_mc[2]
        wordsForCSV.append([republican_mc[0], 1, republican_mc[1]])

      republican_mc = republican_list[(len(republican_list)-2)]
      if (republican_mc, 1) in words:
        words[(republican_mc[0], 1)]['lime_score'].extend(republican_mc[1])
        words[(republican_mc[0], 1)]['position'] = republican_mc[2]
      else:
        words[(republican_mc[0], 1)] = {}
        words[(republican_mc[0], 1)]['lime_score'] = [republican_mc[1]]
        words[(republican_mc[0], 1)]['position'] = republican_mc[2]
        wordsForCSV.append([republican_mc[0], 1, republican_mc[1]])

    elif len(republican_list) == 1:
      republican_mc = republican_list[0]
      if (republican_mc, 1) in words:
        words[(republican_mc[0], 1)]['lime_score'].extend(republican_mc[1])
        words[(republican_mc[0], 1)]['position'] = republican_mc[2]
      else:
        words[(republican_mc[0], 1)] = {}
        words[(republican_mc[0], 1)]['lime_score'] = [republican_mc[1]]
        words[(republican_mc[0], 1)]['position'] = republican_mc[2]
        wordsForCSV.append([republican_mc[0], 1, republican_mc[1]])


  return words, wordsForCSV

In [ ]:
words = {}
wordsForCSV = []

In [ ]:
for tweet in tqdm(tweets, total = len(tweets)):
  words, wordsForCSV = get_max_explained_words(tweet)

  0%|          | 0/86460 [00:00<?, ?it/s]


AttributeError: 'numpy.str_' object has no attribute 'texts_to_sequences'

In [ ]:
import csv

header=["word", "label", "limescore"]

with open('/content/drive/MyDrive/Datasets/Tweets/1_extracted_strong_words_by_bert_base_uncased.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(wordsForCSV)
